## Prerequisites

- Ensure that you have set up your Oracle Cloud Infrastructure (OCI) configuration locally. [[Reference](https://docs.oracle.com/en-us/iaas/Content/API/Concepts/devguidesetupprereq.htm)]
- The user should provide their tenancy OCID to Vision team for access to offline video features.
- Make sure you have the required libraries installed. You can install them by running the below cell.


In [ ]:
# Please use pip/pip3 as per availability
# ! pip3 install oci==2.115.2+preview.1.1678-py2.py3-none-any.whl

: 

## Import Libraries

In [ ]:
import oci
from oci.ai_vision.models import *
from time import slee

## Load OCI config
Set up authentication for OCI by reading configuration from a file and creating a signer instance for secure API communication. The default configuration file location is ```~/.oci/config```.

In [ ]:
config = oci.config.from_file('~/.oci/config', profile_name='ASAGARWA-WORKSHOP')

## Set input variables

In [3]:
ENDPOINT = 'https://vision.aiservice.us-ashburn-1.oci.oraclecloud.com' #service endpoint
COMPARTMENT_ID = "ocid1.compartment.oc1..aaaaaaaa5wmdeu3rf5s4rs4l66rksphne2orz4buauniiqtar63du6ni7icq"
NAMESPACE = "axaemuxiyife" # specify the namespace where the test file is present
BUCKET = "workshopbucket" # specify the bucket name where the test file is present
FILENAME = "mall.mp4" #specify the filename of the test video

## Set output variables

In [4]:
OUTPUT_NAMESPACE = "axaemuxiyife" # specify the namespace where the output has to be stored
OUTPUT_BUCKET = "workshopbucket" # specify the bucket name where the output has to be stored
OUTPUT_PREFIX = "AAGARWA" # specify the prefix where the output has to be stored

## Create AI service vision client

In [10]:
ai_service_vision_client = oci.ai_vision.AIServiceVisionClient(config=config)

## Setup input feature for Offline video features
You can specify the features you want to call. In the below code we are calling all the features

In [6]:
video_object_detection_feature = VideoObjectDetectionFeature()
video_face_detection_feature = VideoFaceDetectionFeature()
video_label_detection_feature = VideoLabelDetectionFeature()
video_text_detection_feature = VideoTextDetectionFeature()
features = [video_text_detection_feature, video_face_detection_feature, video_label_detection_feature, video_object_detection_feature]

## Setup input location

In [7]:
object_location_1 = ObjectLocation()
object_location_1.namespace_name = NAMESPACE
object_location_1.bucket_name = BUCKET
object_location_1.object_name = FILENAME
object_locations = [object_location_1]
input_location = ObjectListInlineInputLocation()
input_location.object_locations = object_locations

## Setup output location

In [8]:
output_location = OutputLocation()
output_location.namespace_name = OUTPUT_NAMESPACE
output_location.bucket_name = OUTPUT_BUCKET
output_location.prefix = OUTPUT_PREFIX

## Create video job

In [11]:
create_video_job_details = CreateVideoJobDetails()
create_video_job_details.features = features
create_video_job_details.compartment_id = COMPARTMENT_ID
create_video_job_details.output_location = output_location
create_video_job_details.input_location = input_location

res = ai_service_vision_client.create_video_job(create_video_job_details=create_video_job_details)

job_id = res.data.id
print(f"Job {job_id} is in {res.data.lifecycle_state} state.")


Job ocid1.aivisionvideojob.oc1.phx.amaaaaaaghwivzaaxjg6y75ap6d737oz22xzakkuqfrcbgans3q5o55egpga is in ACCEPTED state.


## Job in progress
The job progress is tracked till completion with an interval of 5 seconds.

In [12]:
seconds = 0
while res.data.lifecycle_state == "IN_PROGRESS" or res.data.lifecycle_state == "ACCEPTED":
    print(f"Job {job_id} is IN_PROGRESS for {str(seconds)} seconds, progress: {res.data.percent_complete}")
    sleep(5)
    seconds += 5
    res = ai_service_vision_client.get_video_job(video_job_id=job_id)
    
print(f"Job {job_id} is in {res.data.lifecycle_state} state.")

Job ocid1.aivisionvideojob.oc1.phx.amaaaaaaghwivzaaxjg6y75ap6d737oz22xzakkuqfrcbgans3q5o55egpga is IN_PROGRESS for 0 seconds, progress: 0.0
Job ocid1.aivisionvideojob.oc1.phx.amaaaaaaghwivzaaxjg6y75ap6d737oz22xzakkuqfrcbgans3q5o55egpga is IN_PROGRESS for 5 seconds, progress: 0.33333334
Job ocid1.aivisionvideojob.oc1.phx.amaaaaaaghwivzaaxjg6y75ap6d737oz22xzakkuqfrcbgans3q5o55egpga is IN_PROGRESS for 10 seconds, progress: 0.33333334
Job ocid1.aivisionvideojob.oc1.phx.amaaaaaaghwivzaaxjg6y75ap6d737oz22xzakkuqfrcbgans3q5o55egpga is in SUCCEEDED state.


## Get response json from object storage
The output can be found in the output location specified or it can be saved in ```output.json``` file by running the below cell

In [13]:
object_storage_client = oci.object_storage.ObjectStorageClient(config)
object_name = f"{OUTPUT_PREFIX}/{job_id}/{object_location_1.object_name}.json"

video_response = object_storage_client.get_object(OUTPUT_NAMESPACE, OUTPUT_BUCKET, object_name)

file = open('output.json', 'w')
file.write(video_response.data.text)

155717